In [3]:
import torch
import torch.nn as nn
import numpy as np

# Master branch

In [4]:
class MasterNet(nn.Module):
    def __init__(self):
        super(MasterNet, self).__init__()
        
    def forward(self,x):
        return x

# Acoustic Branch

Inputs for acoustic branch will be N x 40 where N [1,33]  
Time step: (2, 10) (seconds?)  
N: relative duration after feature extraction

In [39]:
class AcousticNet(nn.Module):
    def __init__(self, num_conv_layers = 3, kernel_size = 2, conv_width = 32):
        super(AcousticNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=40, out_channels=conv_width, kernel_size=kernel_size), 
            nn.MaxPool1d(kernel_size = 2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=conv_width, out_channels=conv_width, kernel_size=kernel_size), 
            nn.MaxPool1d(kernel_size = 2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=conv_width, out_channels=conv_width, kernel_size=kernel_size), 
            nn.MaxPool1d(kernel_size = 2),
            nn.ReLU()
        )
        self.gru = nn.GRU(input_size=19,hidden_size=32,num_layers=2) # 19 is hardcoded
        self.mean_pool = nn.AvgPool1d(kernel_size=2)
        
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print('after batch', x.shape)
        x = self.conv2(x)
        print('after batch', x.shape)
        x = self.conv3(x)
        print('after batch', x.shape)
        x, _ = self.gru(x)
        print('gru', x.shape)
        x = self.mean_pool(x)
        print('mean pool', x.shape)
        return x

In [40]:
# Test dummy input
net = AcousticNet()
test_vec = torch.randn(10, 40, 17) # samples x features (or channels) x N (relative duration)
output = net(test_vec)
print(f'Shape of output: {output.shape}')
# assert output.shape[-1] == 16

torch.Size([10, 40, 17])
after batch torch.Size([10, 32, 8])
after batch torch.Size([10, 32, 3])
after batch torch.Size([10, 32, 1])


RuntimeError: input.size(-1) must be equal to input_size. Expected 19, got 1

# Lexical Branch

In [5]:
# implement GRU (or transformer)
class LexicalNet(nn.Module):
    def __init__(self, num_gru_layers = 2):
        super(LexicalNet, self).__init__()
        # implement GRU (or transformer)
        self.gru = nn.GRU(input_size=300,hidden_size=32,num_layers=num_gru_layers)
        self.mean_pool = nn.AvgPool1d(kernel_size=2) 
        
    def forward(self, x):
        x, _ = self.gru(x)
        x = self.mean_pool(x)
        return x

In [6]:
# Test dummy input
net = LexicalNet(num_gru_layers = 3)
test_vec = torch.randn(10, 1, 300)
output = net(test_vec)
assert output.shape[-1] == 16